In [1]:
%pip install mlflow scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [2]:
import digitalhub as dh

project = dh.get_or_create_project("demo-mlflow-ml_model-nk")

In [3]:
import os
directory="src"
if not os.path.exists(directory):
    os.makedirs(directory)

In [4]:
%%writefile "src/train-model.py"

from digitalhub_runtime_python import handler
from digitalhub import from_mlflow_run
import mlflow

from sklearn import datasets, svm
from sklearn.model_selection import GridSearchCV

@handler()
def train(project):
    mlflow.sklearn.autolog(log_datasets=True)

    iris = datasets.load_iris()
    parameters = {"kernel": ("linear", "rbf"), "C": [1, 10]}
    svc = svm.SVC()
    clf = GridSearchCV(svc, parameters)

    clf.fit(iris.data, iris.target)
    run_id = mlflow.last_active_run().info.run_id

    # utility to map mlflow run artifacts to model metadata
    model_params = from_mlflow_run(run_id)

    project.log_model(
        name="model-mlflow",
        kind="mlflow",
        **model_params
)

Writing src/train-model.py


In [5]:
train_fn = project.new_function(name="train",
                                kind="python",
                                python_version="PYTHON3_10",
                                code_src="src/train-model.py",
                                handler="train",
                                requirements=["scikit-learn", "mlflow"])

In [6]:
train_run = train_fn.run(action="job", local_execution=False)

In [7]:
model = project.get_model("model-mlflow")
model.spec.path

's3://datalake/demo-mlflow-ml_model-nk/model/model-mlflow/a70a2519-c5de-4b08-bd1e-0732039a302f/'

/v2/models/model/infer

In [8]:
func = project.new_function(name="serve_mlflowmodel",
                            kind="mlflowserve",
                            model_name="mlflow_model",
                            path=model.spec.path + 'model/')

In [9]:
serve_run = func.run(action="serve")

In [10]:
from sklearn import datasets

iris = datasets.load_iris()
data = iris.data[0:2].tolist()
json={
    "inputs": [
        {
        "name": "input-0",
        "shape": [-1, 4],
        "datatype": "FP64",
        "data": data
        }
    ]
}

In [11]:
json

{'inputs': [{'name': 'input-0',
   'shape': [-1, 4],
   'datatype': 'FP64',
   'data': [[5.1, 3.5, 1.4, 0.2], [4.9, 3.0, 1.4, 0.2]]}]}

In [2]:
serve_run = project.get_run(identifier="66f056e7-99b9-481e-858d-d9888df35f91")
serve_run

{'kind': 'mlflowserve+run', 'metadata': {'project': 'demo-mlflow-ml_model-nk', 'name': '66f056e7-99b9-481e-858d-d9888df35f91', 'created': '2024-10-18T09:18:28.424Z', 'updated': '2024-10-18T09:42:40.847Z', 'created_by': 'khurshid@fbk.eu', 'updated_by': 'khurshid@fbk.eu'}, 'spec': {'task': 'mlflowserve+serve://demo-mlflow-ml_model-nk/serve_mlflowmodel:17090442-69cc-4619-b970-3c1bf74b6ed9', 'local_execution': False, 'function': 'mlflowserve://demo-mlflow-ml_model-nk/serve_mlflowmodel:17090442-69cc-4619-b970-3c1bf74b6ed9', 'path': 's3://datalake/demo-mlflow-ml_model-nk/model/model-mlflow/78f7b01b-8f07-4061-9f56-6714037bd906/model/', 'model_name': 'mlflow_model'}, 'status': {'state': 'RUNNING', 'message': 'deployment d-mlflowserveserve-66f056e7-99b9-481e-858d-d9888df35f91 created, service s-mlflowserveserve-66f056e7-99b9-481e-858d-d9888df35f91 created', 'transitions': [{'status': 'RUNNING', 'message': 'deployment d-mlflowserveserve-66f056e7-99b9-481e-858d-d9888df35f91 created, service s-mlf

In [3]:
# serve_run.invoke(model_name="mlflow_model", json=json).json()

In [12]:
service_url = serve_run.refresh().status.to_dict()["service"]["url"]
url = f"http://{service_url}/v2/models/mlflow_model/infer"
url

'http://s-mlflowserveserve-a2cda37c-6eb0-4be4-b93d-1eb9dffdd92d.digitalhub-tenant1:8080/v2/models/mlflow_model/infer'

<!-- /v2/models/model/infer -->

In [19]:
from sklearn import datasets

iris = datasets.load_iris()
data = iris.data[0:2].tolist()
json={
    "inputs": [
        {
        "name": "input-0",
        "shape": [-1, 4],
        "datatype": "FP64",
        "data": data
        }
    ]
}

json

{'inputs': [{'name': 'input-0',
   'shape': [-1, 4],
   'datatype': 'FP64',
   'data': [[5.1, 3.5, 1.4, 0.2], [4.9, 3.0, 1.4, 0.2]]}]}

In [20]:
# url = 's-mlflowserveserve-66f056e7-99b9-481e-858d-d9888df35f91.digitalhub-tenant2:8080'

In [16]:
serve_run.invoke(url=url, json=json).json()
# serve_run.invoke(model_name="mlflow_model", url=url, json=json).json()

{'model_name': 'mlflow_model',
 'id': 'a9e27b4e-92ed-496f-8276-eb4b1fcead0d',
 'parameters': {'content_type': 'np'},
 'outputs': [{'name': 'output-1',
   'shape': [2, 1],
   'datatype': 'INT64',
   'parameters': {'content_type': 'np'},
   'data': [0, 0]}]}